In [4]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime, date

#### Feed the downloaded file here

In [5]:
df = pd.read_csv(r'C:\Users\user\Downloads\transactions2.csv')

In [6]:
df.columns = df.columns.str.strip()
df.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)
0,2022-08-11,5102363071,Quant Active Growth Direct Plan,buy,22.671,441.0636,443.2057,9999.50
1,2022-08-11,5102363071,Quant Infrastructure Growth Direct Plan,buy,432.598,23.1150,23.2289,9999.50
2,2022-08-11,9103116903,PGIM India Midcap Opportunities Growth Direct ...,buy,204.280,48.9500,49.2200,9999.50
3,2022-08-11,17716485234,Canara Robeco Small Cap Growth Direct Plan,buy,388.481,25.7400,25.7100,9999.50
4,2022-08-11,5102363071,Quant Active Growth Direct Plan,buy,11.336,441.0636,443.2057,4999.75


In [7]:
df["Date"] = pd.to_datetime(df["Date"])

In [8]:
df2 = df

#### Prevent all the liquid fund from showing

In [9]:
df = df[~df['Name of the Fund'].str.contains("Liquid|Money|Gilt|Cash|Bond|Debt|Short|Gold|Securities|Duration|Income")]
##remove the upper line in case performance of all the mutual fund is required
df = df.loc[~df['Order'].str.contains("sell")]

In [10]:
today = pd.to_datetime(date.today())

#### Control the number of days to look behind / calculate the Xirr for. for three years we take 365*3 = 1095 days

In [11]:
df = df.loc[df['Date'] > (pd.to_datetime(date.today() - timedelta(days = 1095)))]

In [12]:
mf_name = df['Name of the Fund'].unique()

#### See the list of all the mutual funds

In [13]:
mf_name

array(['Quant Active Growth Direct Plan',
       'Quant Infrastructure Growth Direct Plan',
       'PGIM India Midcap Opportunities Growth Direct Plan',
       'Canara Robeco Small Cap Growth Direct Plan',
       'Quant Tax Growth Direct Plan',
       'Mirae Asset Tax Saver Growth Direct Plan',
       'Axis Small Cap Growth Direct Plan',
       'Mirae Asset Emerging Bluechip Growth Direct Plan',
       'Quant Small Cap Growth Direct Plan',
       'ICICI Prudential Commodities Growth Direct Plan',
       'Tata Digital India Growth Direct Plan',
       'ICICI Prudential Technology Growth Direct Plan'], dtype=object)

In [14]:
df['Amount (INR)'] = df['Amount (INR)']*-1

In [15]:
df.head()

,Date,Folio Number,Name of the Fund,Order,Units,NAV,Current Nav,Amount (INR)
0,2022-08-11,5102363071,Quant Active Growth Direct Plan,buy,22.671,441.0636,443.2057,-9999.50
1,2022-08-11,5102363071,Quant Infrastructure Growth Direct Plan,buy,432.598,23.1150,23.2289,-9999.50
2,2022-08-11,9103116903,PGIM India Midcap Opportunities Growth Direct ...,buy,204.280,48.9500,49.2200,-9999.50
3,2022-08-11,17716485234,Canara Robeco Small Cap Growth Direct Plan,buy,388.481,25.7400,25.7100,-9999.50
4,2022-08-11,5102363071,Quant Active Growth Direct Plan,buy,11.336,441.0636,443.2057,-4999.75


#### Create the  function that calculates Xirr ( funciton copied from https://github.com/SunilVeeravalli/XIRR_in_Python/blob/main/xirr.py )

In [16]:
min_rate = 0
max_rate = 0
def npv(seq_of_rates: np.array, data: pd.DataFrame) -> tuple:
    global min_rate
    global max_rate
    
    for a_rate in seq_of_rates:
        max_date = data['Date'].max()
        data['npv'] = data['Amount'] * ((1 + (a_rate / 100)) ** ((max_date - data['Date']).dt.days / 365))
        
        if data['Amount'].sum() > 0:
            if data['npv'].sum() > 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
        else:
            if data['npv'].sum() < 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
    
    return min_rate, max_rate


def xirr(data: pd.DataFrame) -> float:
    # Finding out which column contains Date and Amount
    try:
        data.iloc[:, 0].astype(float)
        col_names = ['Amount', 'Date']
    except Exception as _:
        col_names = ['Date', 'Amount']
    
    # Renaming the columns accordingly and converting to correct data types
    data.columns = col_names
    data = data.assign(Date = pd.to_datetime(arg = data['Date'], infer_datetime_format = True, dayfirst = True).dt.date,
                       Amount = data['Amount'].astype(float))
    
    if data['Amount'].sum() > 0:
        step_values = [100, 10, 1, 0.01, 0.001, 0.0001]
        stop = 10000
    else:
        step_values = [-100, -10, -1, -0.01, -0.001, -0.0001]
        stop = -10000
    
    start = 0
    for i in range(len(step_values)):
        seq_of_rates = np.arange(start = start, stop = stop, step = step_values[i])
        start, stop = npv(seq_of_rates, data)
    
    return (start + stop) / 2


In [17]:
solution = []
mf= []
for i in range(0,len(mf_name)):
    mf = df[df['Name of the Fund'] == mf_name[i]]
    mf = mf.append(pd.DataFrame({'Date': today, 'Amount (INR)':mf.Units.sum() * mf['Current Nav'].unique() }), 
                   ignore_index = True, sort = True)
    ready_df = mf[['Date','Amount (INR)']]
    solution.append(xirr(ready_df))

In [18]:
final_df = pd.DataFrame(list(zip(mf_name.tolist(),solution)))

In [19]:
final_df.columns = ['Name of the Fund', 'Xirr']

### See the final performance of the mutual fund sorted based on the overall Xirr

In [20]:
final_df.sort_values('Xirr', ascending = False)

,Name of the Fund,Xirr
9,ICICI Prudential Commodities Growth Direct Plan,132.99885
6,Axis Small Cap Growth Direct Plan,93.99385
8,Quant Small Cap Growth Direct Plan,91.59965
7,Mirae Asset Emerging Bluechip Growth Direct Plan,89.03985
4,Quant Tax Growth Direct Plan,88.85985
10,Tata Digital India Growth Direct Plan,86.82625
5,Mirae Asset Tax Saver Growth Direct Plan,84.69905
1,Quant Infrastructure Growth Direct Plan,83.88635
2,PGIM India Midcap Opportunities Growth Direct ...,81.43245
0,Quant Active Growth Direct Plan,68.75435
